# RevenueBase-THT

## Get WARC

In [6]:
# Set the Index URL for the CDXJ Index Server API
base_url = "https://index.commoncrawl.org"
crawl_index_url = "CC-MAIN-2022-49-index"
index_url = urljoin(base_url, crawl_index_url)

# WARC database
database_url = 'https://data.commoncrawl.org/'

In [20]:
# Save a pickle file of the URLs index for a domain as a list of json files (results_json=domain_locations variable downstream) [DOMAIN>URLs: 1>MANY]

def get_index_request(index_url, domain_url, pickle_name):
    # set headers
    headers = {'Accept': 'application/json'}
    # Set the parameters for the request
    params = {
        "url": domain_url,  # The domain URL to search for
        "output": "json",  # The format of the results
        "limit": "10000"  # The maximum number of results
    }

    # Send the request to the CDXJ Server API
    response = requests.get(index_url, headers=headers, params=params, allow_redirects=False)
    print("Respone file type is ", response.headers["Content-Type"])
    print(domain_url)
    
#     pickle_name = f'domain_index_json_list_{domain_url}.pkl'
   
    # Check the status code of the response
    if response.status_code == 200:
        # The request was successful, process the results to a list of jsons
        results = response.content.decode().split('\n')
        results_json = []
        for json_obj in results:
            if json_obj:
                results_json.append(json.loads(json_obj))
        print(results_json)
        # pickle file - containing location info for all urls retrieved per domain
        with open(pickle_name, 'wb') as f:
            pickle.dump(results_json, f)
            print(f"Saved as {pickle_name}.pkl")
    else:
        # There was an error with the request
        error_dict = [{'urlkey': f"Error: {response.status_code}", 
            'status': '200', 
            'length': '912',
            'offset': '2546937', 
            'filename': 'crawl-data/CC-MAIN-2022-49/segments/1669446710534.53/crawldiagnostics/CC-MAIN-20221128171516-20221128201516-00783.warc.gz', 
           }]
        with open(pickle_name, 'wb') as f:
            pickle.dump(error_dict, f)
            print(f"Saved as {pickle_name}.pkl")
        print(f"Error: {response.status_code}")
        
    time.sleep(3)

In [9]:
get_index_request(index_url, domain_url, pickle_name)

Respone file type is  text/x-ndjson
blogspot.com
[{'urlkey': 'com,blogspot)/', 'timestamp': '20221128174621', 'url': 'http://www.blogspot.com', 'mime': 'text/html', 'mime-detected': 'text/html', 'status': '302', 'digest': '2TDTFYDDBL6RMP6RYZSSKM7CYDOCWBCM', 'length': '912', 'offset': '2546937', 'filename': 'crawl-data/CC-MAIN-2022-49/segments/1669446710534.53/crawldiagnostics/CC-MAIN-20221128171516-20221128201516-00783.warc.gz', 'redirect': 'https://www.blogger.com'}, {'urlkey': 'com,blogspot)/', 'timestamp': '20221130194410', 'url': 'http://www.blogspot.com', 'mime': 'text/html', 'mime-detected': 'text/html', 'status': '302', 'digest': '2TDTFYDDBL6RMP6RYZSSKM7CYDOCWBCM', 'length': '915', 'offset': '2434300', 'filename': 'crawl-data/CC-MAIN-2022-49/segments/1669446710771.39/crawldiagnostics/CC-MAIN-20221130192708-20221130222708-00783.warc.gz', 'redirect': 'https://www.blogger.com'}, {'urlkey': 'com,blogspot)/', 'timestamp': '20221201235209', 'url': 'https://www.blogspot.com/', 'mime': 

In [8]:
# load saved index list
def load_index_list(pkl_file):
    try:
        with open(pkl_file, 'rb') as f:
            domain_index_json_list = pickle.load(f)
    except:
        pass
    return domain_index_json_list

In [10]:
def clean_301_urls(domain_locations):
    for urlkey in domain_locations:
        try:
            domain_locations_1 = [urlkey for urlkey in domain_locations if urlkey['status'] == '200']
        except:
            pass
    return domain_locations_1

### Get WARC Files

In [13]:
# get warc file per URL [Going from JSONs level to URL Level, URL>WARC: 1=1]
    
def get_warc_request(domain_locations): #using only the first response (not quite sure why there are many index places for one url, for example for wordpress.com there are 69 including the 301s)
    
    url = domain_locations[0]
    # location of domain
    warc_record_length = int(url['length'])
    warc_record_offset = int(url['offset'])
    filename = url['filename']

    # url construction per domain
    domain_url = urljoin(database_url, filename)

    # streaming a WARC file from HTTPS and printing HTML code (DEFINE AS FUNCTION)
    response = requests.get(domain_url, 
                            headers={'Range': f'bytes={warc_record_offset}-{warc_record_offset + warc_record_length - 1}'},
                            stream=True)
    
    time.sleep(3)
    
    return response           

In [14]:
# Iterates over url_list and saves all the responses into the list all_pages, 
# which is stored to the response.pickle file.

def save_response(response_warc, domain_url):
    
    all_pages = []
    # for url in url_list:
    all_pages.append(response_warc)

    with open(f"responses_{domain_url}.pickle", "wb") as f:
        pickle.dump(all_pages, f)

In [58]:
# Complete function

def get_html(index_url, database_url, domain_list):
    
    for domain in domain_list:
        # define index_url name
        pickle_name = f'domain_index_json_list_{domain}.pkl'
        # define warc response name
        warc_pickle_name = f"responses_{domain}.pickle"

        # get request
        get_index_request(index_url=index_url, domain_url=domain, pickle_name=pickle_name)
        
        # load request
        domain_locations = load_index_list(pickle_name)

        # clean warc
        domain_locations = clean_301_urls(domain_locations)
        print(domain_locations)
        
        # get warc
        response_warc = get_warc_request(domain_locations)
       
        # save warc request
        save_response(response_warc=response_warc, domain_url=domain)

In [59]:
get_html(index_url, database_url, domain_list)

Respone file type is  text/x-ndjson
blogspot.com
[{'urlkey': 'com,blogspot)/', 'timestamp': '20221128174621', 'url': 'http://www.blogspot.com', 'mime': 'text/html', 'mime-detected': 'text/html', 'status': '302', 'digest': '2TDTFYDDBL6RMP6RYZSSKM7CYDOCWBCM', 'length': '912', 'offset': '2546937', 'filename': 'crawl-data/CC-MAIN-2022-49/segments/1669446710534.53/crawldiagnostics/CC-MAIN-20221128171516-20221128201516-00783.warc.gz', 'redirect': 'https://www.blogger.com'}, {'urlkey': 'com,blogspot)/', 'timestamp': '20221130194410', 'url': 'http://www.blogspot.com', 'mime': 'text/html', 'mime-detected': 'text/html', 'status': '302', 'digest': '2TDTFYDDBL6RMP6RYZSSKM7CYDOCWBCM', 'length': '915', 'offset': '2434300', 'filename': 'crawl-data/CC-MAIN-2022-49/segments/1669446710771.39/crawldiagnostics/CC-MAIN-20221130192708-20221130222708-00783.warc.gz', 'redirect': 'https://www.blogger.com'}, {'urlkey': 'com,blogspot)/', 'timestamp': '20221201235209', 'url': 'https://www.blogspot.com/', 'mime': 

___________CLEAAAAANEEEDDD LOCATIONS___________________
[{'urlkey': 'com,wordpress)/', 'timestamp': '20221126095628', 'url': 'https://wordpress.com/', 'mime': 'text/html', 'mime-detected': 'text/html', 'status': '200', 'digest': 'LLQUZVTKBSDGX5IZQWNE6NLJ6BH3X4DE', 'length': '44912', 'offset': '666495313', 'filename': 'crawl-data/CC-MAIN-2022-49/segments/1669446706285.92/warc/CC-MAIN-20221126080725-20221126110725-00547.warc.gz', 'languages': 'eng', 'encoding': 'UTF-8'}, {'urlkey': 'com,wordpress)/', 'timestamp': '20221126164628', 'url': 'https://wordpress.com/', 'mime': 'text/html', 'mime-detected': 'text/html', 'status': '200', 'digest': 'LLQUZVTKBSDGX5IZQWNE6NLJ6BH3X4DE', 'length': '44915', 'offset': '647871013', 'filename': 'crawl-data/CC-MAIN-2022-49/segments/1669446708010.98/warc/CC-MAIN-20221126144448-20221126174448-00547.warc.gz', 'languages': 'eng', 'encoding': 'UTF-8'}, {'urlkey': 'com,wordpress)/', 'timestamp': '20221127052230', 'url': 'https://wordpress.com/', 'mime': 'text/h

Respone file type is  text/html
wikipedia.org
Saved as domain_index_json_list_wikipedia.org.pkl.pkl
Error: 504
___________CLEAAAAANEEEDDD LOCATIONS___________________
[{'urlkey': 'Error: 504', 'status': '200', 'length': '912', 'offset': '2546937', 'filename': 'crawl-data/CC-MAIN-2022-49/segments/1669446710534.53/crawldiagnostics/CC-MAIN-20221128171516-20221128201516-00783.warc.gz'}]


KeyboardInterrupt: 